In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from janitor import clean_names

sns.set_theme()  # plotting theme

In [2]:
# read raw data
raw_data = gpd.read_file('data/raw_data.geojson')

In [3]:
# map 'intake_is_dead' to integers (0 or 1)
raw_data['intake_is_dead'] = raw_data['intake_is_dead'].mask(
    raw_data['intake_is_dead'] == 'Alive on Intake', 0
)

In [4]:
raw_data = raw_data.astype(
    {
        'intake_is_dead': 'bool',
        'was_outcome_alive': 'bool',
        'is_current_month': 'bool',
        'animal_type': 'category',
        'primary_color': 'category',
        'secondary_color': 'category',
        'sex': 'category',
        'intake_cond': 'category',
        'intake_type': 'category',
        'intake_subtype': 'category',
        'reason': 'category',
        'jurisdiction': 'category',
        'outcome_type': 'category',
        'outcome_subtype': 'category',
    }
)

In [ ]:
# mapping for raw outcome_type to grouped category
outcome_mapping = {
    # 1. Adopted
    "ADOPTION": "Adopted",
    "FOSTER TO ADOPT": "Adopted",
    "FOSTER": "Adopted",
    "HOMEFIRST": "Adopted",
    # 2. Transferred/Rescued
    "RESCUE": "Transferred/Rescued",
    "TRANSFER": "Transferred/Rescued",
    "TRANSPORT": "Transferred/Rescued",
    "RETURN TO RESCUE": "Transferred/Rescued",
    # 3. Returned/Community Programs
    "RETURN TO OWNER": "Returned/Community",
    "SHELTER, NEUTER, RETURN": "Returned/Community",
    "COMMUNITY CAT": "Returned/Community Programs",
    "RETURN TO WILD HABITAT": "Returned/Community",
    "TRAP, NEUTER, RELEASE": "Returned/Community",
    # 4. Negative Outcomes
    "EUTHANASIA": "Negative Outcome",
    "DIED": "Negative Outcome",
    "DISPOSAL": "Negative Outcome",
    # 5. Other/Uncertain
    "MISSING": "Other/Uncertain",
    "DUPLICATE": "Other/Uncertain",
}

In [9]:
raw_data["outcome_grouped"] = raw_data["outcome_type"].map(outcome_mapping)

In [10]:
raw_data.head()

,animal_id,animal_name,animal_type,primary_color,secondary_color,sex,dob,intake_date,intake_cond,intake_type,...,outcome_subtype,latitude,longitude,intake_is_dead,outcome_is_dead,was_outcome_alive,intake_duration,is_current_month,geometry,outcome_grouped
0,A668536,LUCKY,DOG,BLACK,WHITE,Male,2012-12-29,2021-12-26,NORMAL,STRAY,...,WALKIN,33.873055,-118.152245,False,FALSE,True,3.0,True,POINT (-118.15225 33.87305),Returned/Community
1,A688134,*CANARY,CAT,GRAY TABBY,NaN,Female,2017-11-07,2022-11-07,INJURED SEVERE,STRAY,...,STRAYCATAL,33.873055,-118.152245,False,FALSE,True,16.0,True,POINT (-118.15225 33.87305),Transferred/Rescued
2,A668537,ROXY,DOG,BLACK,WHITE,Female,2012-12-29,2021-12-26,NORMAL,STRAY,...,WALKIN,33.873055,-118.152245,False,FALSE,True,3.0,True,POINT (-118.15225 33.87305),Returned/Community
3,A605848,NO NAME,CAT,BLACK,NaN,Male,2011-04-06,2018-04-06,ILL SEVERE,CONFISCATE,...,ILL SEVERE,33.872790,-118.179213,False,TRUE,False,2.0,True,POINT (-118.17921 33.87279),Negative Outcomes
4,A605853,ASTERIA,CAT,TORTIE,NaN,Female,2010-04-19,2018-04-06,ILL SEVERE,CONFISCATE,...,SPCALA,33.872790,-118.179213,False,FALSE,True,141.0,True,POINT (-118.17921 33.87279),Transferred/Rescued


In [12]:
# outcome grouped var to category
raw_data = raw_data.astype({'outcome_grouped': 'category'})

In [15]:
# age at intake (in days)
age_days = (raw_data['intake_date'] - raw_data['dob']).dt.days

# age at intake (in years)
raw_data['age_at_intake'] = age_days / 365.25

In [16]:
raw_data.head()

,animal_id,animal_name,animal_type,primary_color,secondary_color,sex,dob,intake_date,intake_cond,intake_type,...,latitude,longitude,intake_is_dead,outcome_is_dead,was_outcome_alive,intake_duration,is_current_month,geometry,outcome_grouped,age_at_intake
0,A668536,LUCKY,DOG,BLACK,WHITE,Male,2012-12-29,2021-12-26,NORMAL,STRAY,...,33.873055,-118.152245,False,FALSE,True,3.0,True,POINT (-118.15225 33.87305),Returned/Community,8.991102
1,A688134,*CANARY,CAT,GRAY TABBY,NaN,Female,2017-11-07,2022-11-07,INJURED SEVERE,STRAY,...,33.873055,-118.152245,False,FALSE,True,16.0,True,POINT (-118.15225 33.87305),Transferred/Rescued,4.999316
2,A668537,ROXY,DOG,BLACK,WHITE,Female,2012-12-29,2021-12-26,NORMAL,STRAY,...,33.873055,-118.152245,False,FALSE,True,3.0,True,POINT (-118.15225 33.87305),Returned/Community,8.991102
3,A605848,NO NAME,CAT,BLACK,NaN,Male,2011-04-06,2018-04-06,ILL SEVERE,CONFISCATE,...,33.872790,-118.179213,False,TRUE,False,2.0,True,POINT (-118.17921 33.87279),Negative Outcomes,7.000684
4,A605853,ASTERIA,CAT,TORTIE,NaN,Female,2010-04-19,2018-04-06,ILL SEVERE,CONFISCATE,...,33.872790,-118.179213,False,FALSE,True,141.0,True,POINT (-118.17921 33.87279),Transferred/Rescued,7.964408
